In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.optimization as opt
import biogeme.version as ver
from biogeme.expressions import Beta, Variable

We first print the version of Biogeme that has been used to generate the results.

In [2]:
print(ver.getText())

biogeme 3.2.8 [2021-07-29]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



# Sampling procedures

Load the population and display information about it.

In [3]:
url = ('https://raw.githubusercontent.com/michelbierlaire/mooc-discrete-choice/master/'
       'syntheticPopulationWithChoice.zip')
population = pd.read_csv(url)

In [4]:
population.describe()

,Id,MarginalCostPT,WaitingTimePT,CostCarCHF,NbTransf,distance_km,TimePT,TimeCar,OccupStat,LangCode,CarAvail,Education,TripPurpose,Prob0,Prob1,Prob2,Choice
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000
mean,499999.500000,11.130081,13.125910,5.760960,2.009250,40.380471,107.915374,40.702842,1.923853,1.744393,1.102572,4.153050,1.656476,0.286302,0.649073,6.462583e-02,0.779402
std,288675.278932,16.310957,22.341342,8.404421,2.200499,63.054669,88.125821,48.109134,0.869789,0.436202,0.441161,1.518441,0.474885,0.283246,0.271691,1.078408e-01,0.549619
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,3.000000,1.000000,0.000002,0.000000,5.856389e-111,0.000000
25%,249999.750000,2.533611,0.000000,1.451197,0.000000,8.844138,46.343927,13.346889,1.000000,1.000000,1.000000,3.000000,1.000000,0.084735,0.537786,2.228475e-06,0.000000
50%,499999.500000,5.473843,5.212542,2.969225,1.621549,18.861824,85.299806,26.206421,2.000000,2.000000,1.000000,3.000000,2.000000,0.181506,0.710326,4.427529e-03,1.000000
75%,749999.250000,12.756591,17.591064,6.292116,3.281843,43.464997,143.598715,49.411055,3.000000,2.000000,1.000000,6.000000,2.000000,0.368227,0.855592,8.545803e-02,1.000000
max,999999.000000,275.905800,470.223169,81.148874,16.785921,622.584149,893.873634,592.327680,3.000000,2.000000,3.000000,7.000000,2.000000,1.000000,0.999997,9.720215e-01,2.000000


In [5]:
populationSize = population.shape[0]

We set the seed so that the results are reproducible

In [6]:
np.random.seed(seed=9021967)

Generic stratified sampling procedure

In [7]:
def stratifiedSample(name, mask, H, sampleSize=1000):
    # name: name of the sampling scheme
    # mask: list of masks identifying the strata
    # H: target shares for each stratum
    groupname = f'{name}Group'
    # Calculate the share of each group in the population
    W = [None] * len(mask)
    for i, m in enumerate(mask):
        W[i] = population[m].shape[0] / populationSize
        population.loc[m, groupname] = i
        population.loc[m, f'Weight{name}'] = W[i] / H[i] if H[i] != 0 else 0
    # Sampling
    groupsize = np.array(H) * sampleSize
    
    def sampleStratum(x):
        size = int(groupsize[int(x[groupname].mean())])
        if size == 0:
            return None
        elif size > x.shape[0]:
            # not enough individual in stratum to reach the requirements
            return x
        else:
            return x.sample(n=size)
    # The statement 'int(x[groupname].mean())' retireves the index of the group.
    sample = (population.groupby(groupname).
              apply(sampleStratum).
              reset_index(drop=True))
    population.drop(groupname, axis='columns', inplace=True)
    return sample

## Endogenously stratified Sample

The strata are defined based on car availability and the chosen alternative.  As the choice is involved in the definition of strata, the stratified sampling is *endogenous*. Target shares in the sample: there are 5 non empty strata, taking 20% each

In [8]:
def ess1(sampleSize=1000):
    ess1Mask = [(population['CarAvail'] == i) & (population['Choice'] == j) 
                for i in [1, 3] for j in [0, 1, 2]]
    H = [0.2, 0.2, 0.2, 0.2, 0.0, 0.2]
    name = 'ESS1'
    return name, stratifiedSample(name, ess1Mask, H, sampleSize)

# Choice model

The specification of the choice model is available in the file <code>spec_optima</code>. Note that it is the exact same specification that has been used to generate the synthetic choices.

If the file ``spec_optima.py``is in your local directory, simply use the following statement to obtain the model specification. For this notebook running on a remote server, we have copied the code in the following cell.

In [9]:
#from spec_optima import V, av, Choice

In [10]:
# File spec_optima.py
#
# Definition of the variables in the data file
Id = Variable('Id')
MarginalCostPT = Variable('MarginalCostPT')
WaitingTimePT = Variable('WaitingTimePT')
CostCarCHF = Variable('CostCarCHF')
NbTransf = Variable('NbTransf')
distance_km = Variable('distance_km')
TimePT = Variable('TimePT')
TimeCar = Variable('TimeCar')
OccupStat = Variable('OccupStat')
LangCode = Variable('LangCode')
CarAvail = Variable('CarAvail')
Education = Variable('Education')
TripPurpose = Variable('TripPurpose')
Prob0 = Variable('Prob0')
Prob1 = Variable('Prob1')
Prob2 = Variable('Prob2')
Choice = Variable('Choice')

TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5

ASC_PT_GERMAN_CARAVAIL = Beta('ASC_PT_GERMAN_CARAVAIL', 0, None, None, 0)

ASC_PT_GERMAN_NOCARAVAIL = Beta('ASC_PT_GERMAN_NOCARAVAIL', 0, None, None, 0)

ASC_PT_GERMAN = ASC_PT_GERMAN_CARAVAIL * (
    CarAvail == 1
) + ASC_PT_GERMAN_NOCARAVAIL * (CarAvail != 1)

ASC_PT_FRENCH_CARAVAIL = Beta('ASC_PT_FRENCH_CARAVAIL', 0, None, None, 0)

ASC_PT_FRENCH_NOCARAVAIL = Beta('ASC_PT_FRENCH_NOCARAVAIL', 0, None, None, 0)

ASC_PT_FRENCH = ASC_PT_FRENCH_CARAVAIL * (
    CarAvail == 1
) + ASC_PT_FRENCH_NOCARAVAIL * (CarAvail != 1)

ASC_PT = ASC_PT_GERMAN * (LangCode != 1) + ASC_PT_FRENCH * (LangCode == 1)

ASC_CAR_GERMAN = Beta('ASC_CAR_GERMAN', 0, None, None, 0)
ASC_CAR_FRENCH = Beta('ASC_CAR_FRENCH', 0, None, None, 0)

ASC_CAR = ASC_CAR_FRENCH * (LangCode == 1) + ASC_CAR_GERMAN * (LangCode != 1)

BETA_TIME_FULL = Beta('BETA_TIME_FULL', 0, None, None, 0)

BETA_TIME_PARTTIME = Beta('BETA_TIME_PARTTIME', 0, None, None, 0)

BETA_TIME_OTHERS = Beta('BETA_TIME_OTHERS', 0, None, None, 0)

BETA_TIME = (
    BETA_TIME_FULL * (OccupStat == 1)
    + BETA_TIME_PARTTIME * (OccupStat == 2)
    + BETA_TIME_OTHERS * (OccupStat == 3)
)

BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_CAR = Beta('BETA_COST_CAR', 0, None, None, 0)

BETA_WAITING_WORK = Beta('BETA_WAITING_WORK', 0, None, None, 0)

BETA_WAITING_NONWORK = Beta('BETA_WAITING_NONWORK', 0, None, None, 0)

BETA_WAITING = BETA_WAITING_WORK * (
    TripPurpose == 1
) + BETA_WAITING_NONWORK * (TripPurpose != 1)

BETA_DIST_VOCA = Beta('BETA_DIST_VOCA', 0, None, None, 0)
BETA_DIST_HIGH_SCHOOL = Beta('BETA_DIST_HIGH_SCHOOL', 0, None, None, 0)

BETA_DIST_HIGHER = Beta('BETA_DIST_HIGHER', 0, None, None, 0)
BETA_DIST_UNIV = Beta('BETA_DIST_UNIV', 0, None, None, 0)

BETA_DIST = (
    BETA_DIST_VOCA * (Education == 3)
    + BETA_DIST_HIGH_SCHOOL * (Education == 4)
    + BETA_DIST_HIGHER * (Education == 6)
    + BETA_DIST_UNIV * (Education == 7)
)

LAMBDA_COST = 0.3214999879822265

V_PT = (
    ASC_PT
    + BETA_TIME * TimePT_scaled
    + BETA_COST_PT * models.boxcox(MarginalCostPT_scaled, LAMBDA_COST)
    + BETA_WAITING * WaitingTimePT ** 0.5
)

V_CAR = (
    ASC_CAR
    + BETA_TIME * TimeCar_scaled
    + BETA_COST_CAR * models.boxcox(CostCarCHF_scaled, LAMBDA_COST)
)

V_SM = BETA_DIST * distance_km_scaled


V = {0: V_PT, 1: V_CAR, 2: V_SM}

av = {0: 1, 1: CarAvail != 3, 2: 1}



For each sample, we estimate the parameters once with ESML, and once with WESML

In [11]:
def sampleAndEstimate(sampling):
    name, sample = sampling()
    database = db.Database(name, sample)
    logprob = models.loglogit(V, av, Choice)
    formulas = {'loglike': logprob, 'weight': Variable(f'Weight{name}')}
    biogeme_noweight = bio.BIOGEME(database, logprob)
    biogeme_noweight.modelName = name
    biogeme_noweight.generateHtml = False
    biogeme_noweight.generatePickle = False
    results_noweight = biogeme_noweight.quickEstimate(algoParameters={'maxiter': 100})
    pandasResults_noweight = results_noweight.getEstimatedParameters()

    biogeme_weight = bio.BIOGEME(database, formulas)
    biogeme_weight.modelName = name
    biogeme_weight.generateHtml = False
    biogeme_weight.generatePickle = False
    results_weight = biogeme_weight.quickEstimate(algoParameters={'maxiter': 100})
    pandasResults_weight = results_weight.getEstimatedParameters()
    return pandasResults_noweight['Value'].T, pandasResults_weight['Value'].T

# Code the experiment

We first load the true values of the parameters

In [12]:
trueValuesOfTheParameters = {'ASC_CAR_FRENCH': -2.3964170553344726,
 'ASC_CAR_GERMAN': -3.5077163185270175,
 'ASC_PT_FRENCH_CARAVAIL': -2.361224966506705,
 'ASC_PT_FRENCH_NOCARAVAIL': 4.620997760623915,
 'ASC_PT_GERMAN_CARAVAIL': -2.3439305816983027,
 'ASC_PT_GERMAN_NOCARAVAIL': 2.4527259821306058,
 'BETA_COST_CAR': -1.6857507208148164,
 'BETA_COST_PT': -0.6736462504002271,
 'BETA_DIST_HIGHER': -1.6393574970002036,
 'BETA_DIST_HIGH_SCHOOL': -1.6452607972992284,
 'BETA_DIST_UNIV': -0.9390891835029462,
 'BETA_DIST_VOCA': -2.065461231848294,
 'BETA_TIME_FULL': -2.8832500848406823,
 'BETA_TIME_OTHERS': -0.16765495123858037,
 'BETA_TIME_PARTTIME': -2.0805822527972277,
 'BETA_WAITING_NONWORK': -0.2650336860286495,
 'BETA_WAITING_WORK': -0.1281619517257249}

The following procedure implements the experiment. It repeats sampling and estimation, stores all estimation results. When it is done, the mean and the standard deviation is calculated for each parameter, as well as the *t*-statistic. 

In [13]:
def runExperiment(sampling, repetitions):
    results_noweight = pd.DataFrame(columns=trueValuesOfTheParameters)
    results_weight = pd.DataFrame(columns=trueValuesOfTheParameters)
    for i in range(repetitions):
        if i % 20 == 0:
            print(f'Repetition {i}/{repetitions}')
        row_noweight, row_weight = sampleAndEstimate(sampling)
        results_noweight = results_noweight.append(row_noweight)
        results_weight = results_weight.append(row_weight)


    comparisons = pd.DataFrame.from_dict(trueValuesOfTheParameters, orient='index', columns=['True'])
    comparisons['Estimated_noweight'] = results_noweight.mean()
    comparisons['StdDev_noweight'] = results_noweight.std()
    comparisons['t-test_noweight'] = ((comparisons['Estimated_noweight'] - comparisons['True']) 
                                      / comparisons['StdDev_noweight'])
    comparisons['Estimated_weight'] = results_weight.mean()
    comparisons['StdDev_weight'] = results_weight.std()
    comparisons['t-test_weight'] = ((comparisons['Estimated_weight'] - comparisons['True']) 
                                    / comparisons['StdDev_weight'])
    return results_noweight, results_weight, comparisons

# Run the experiment

In [14]:
repetitions = 200

## Endogenous sampling

In [15]:
%%time
resultsESS1_noweight, resultsESS1_weight, comparisonsESS1 = runExperiment(ess1, repetitions=repetitions)

Repetition 0/200
Repetition 20/200
Repetition 40/200
Repetition 60/200
Repetition 80/200
Repetition 100/200
Repetition 120/200
Repetition 140/200
Repetition 160/200
Repetition 180/200
CPU times: user 14min 13s, sys: 35.1 s, total: 14min 48s
Wall time: 3min 19s


### Correct the constants

Calculate the shares in the population and the sample

In [16]:
ess1Mask = [(population['CarAvail'] == i) & (population['Choice'] == j) 
                for i in [1, 3] for j in [0, 1, 2]]
W = [population[m].shape[0] / populationSize for m in ess1Mask]
H = [0.2, 0.2, 0.2, 0.2, 0.0, 0.2]

Group numbering: the next statement displays the numbering of the groups. For instance, group 0 corresponds to ``CarAvail = 1``and ``Choice = 0``, and group 4 corresponds to ``CarAvail = 3`` and ``Choice = 1``.

In [17]:
{k: v for k, v in enumerate([(i, j) for i in [1, 3] for j in [0, 1, 2]])}

{0: (1, 0), 1: (1, 1), 2: (1, 2), 3: (3, 0), 4: (3, 1), 5: (3, 2)}

The first index is the group associated with the constant. The second index is the corresponding group for "Slow modes", used as a reference.

In [18]:
groups = {'ASC_CAR_FRENCH': (1, 2), 
          'ASC_CAR_GERMAN': (1, 2), 
          'ASC_PT_FRENCH_CARAVAIL': (0, 2), 
          'ASC_PT_FRENCH_NOCARAVAIL': (3, 5), 
          'ASC_PT_GERMAN_CARAVAIL': (0, 2), 
          'ASC_PT_GERMAN_NOCARAVAIL': (3, 5)}

Copy the estimated values in a new column, and correct the constants.

In [19]:
comparisonsESS1['Corrected_noweight'] = comparisonsESS1['Estimated_noweight']
for k, (i, j) in groups.items():
    comparisonsESS1.loc[k, 'Corrected_noweight'] = (comparisonsESS1.loc[k, 'Estimated_noweight'] 
                                                    - np.log(H[i]) + np.log(W[i]) 
                                                    + np.log(H[j]) - np.log(W[j]))


Recalculate the $t$-test with the corrected constants

In [20]:
comparisonsESS1['t-test_noweight'] = (comparisonsESS1['Corrected_noweight'] - comparisonsESS1['True']) / comparisonsESS1['StdDev_noweight']

In [21]:
comparisonsESS1

,True,Estimated_noweight,StdDev_noweight,t-test_noweight,Estimated_weight,StdDev_weight,t-test_weight,Corrected_noweight
ASC_CAR_FRENCH,-2.396417,-4.961063,0.480341,-0.538551,-2.615858,0.603873,-0.363389,-2.655105
ASC_CAR_GERMAN,-3.507716,-6.126763,0.430973,-0.726469,-3.754314,0.562784,-0.438176,-3.820805
ASC_PT_FRENCH_CARAVAIL,-2.361225,-3.863984,0.447510,-0.478431,-2.554087,0.545594,-0.353489,-2.575328
ASC_PT_FRENCH_NOCARAVAIL,4.620998,-0.656104,0.834493,-0.409499,3.907046,1.153505,-0.618942,4.279274
ASC_PT_GERMAN_CARAVAIL,-2.343931,-3.822051,0.276156,-0.686076,-2.477502,0.364243,-0.366708,-2.533395
ASC_PT_GERMAN_NOCARAVAIL,2.452726,-2.610609,0.220315,-0.580792,2.342492,0.396771,-0.277828,2.324769
BETA_COST_CAR,-1.685751,-1.774996,0.204863,-0.435633,-1.774852,0.261212,-0.341106,-1.774996
BETA_COST_PT,-0.673646,-0.686131,0.088471,-0.141121,-0.702658,0.118678,-0.244455,-0.686131
BETA_DIST_HIGHER,-1.639357,-1.761969,0.173985,-0.704722,-1.723016,0.215809,-0.387652,-1.761969
BETA_DIST_HIGH_SCHOOL,-1.645261,-1.758491,0.174174,-0.650099,-1.714282,0.242671,-0.284421,-1.758491


In the above table, we see that both estimators are in good agreement with the true values. In theory, the precision is higher when ESML is used, and the constants are corrected. It can be seen empirically by calculating the average standard errors in both cases. The value without weight should be lower than the value with weights.

In [26]:
comparisonsESS1['StdDev_noweight'].abs().mean()

0.3405472529193685

In [27]:
comparisonsESS1['StdDev_weight'].abs().mean()

0.4445028223510333